### Summarize field data

In [1]:
import sys, os
import pandas as pd
import numpy as np

In [2]:
#Read the file
theFN = '../scratch/beetle/AllFieldData.csv'
outFN = '../scratch/BeetleFieldSummary.csv'
df = pd.read_csv(theFN)

In [3]:
df.columns

Index([u'uid', u'domainID', u'siteID', u'plotID', u'trapID', u'nlcdClass',
       u'decimalLatitude', u'decimalLongitude', u'geodeticDatum',
       u'coordinateUncertainty', u'elevation', u'elevationUncertainty',
       u'setDate', u'collectDate', u'boutNumber', u'eventID', u'sampleID',
       u'daysOfTrapping', u'cupStatus', u'lidStatus', u'fluidLevel',
       u'trapReset', u'samplingProtocol', u'recordedBy', u'remarks',
       u'missingRecordsPerBoutQF', u'duplicateCollectionEventQF',
       u'compareSetCollectDateQF'],
      dtype='object')

In [4]:
#Summarize static plot attributes into a data frame
grp = df.groupby(by='plotID')
plotAttributes = grp.agg({'nlcdClass':'first',
                          'decimalLatitude':'mean',
                          'decimalLongitude':'mean',
                          'elevation':'mean'
                         })
plotAttributes.head()

,decimalLongitude,elevation,decimalLatitude,nlcdClass
plotID,,,,
BART_002,-71.272850,550.8,44.035081,deciduousForest
BART_018,-71.277857,315.5,44.058474,mixedForest
BART_025,-71.313142,542.8,44.059386,evergreenForest
BART_028,-71.306532,485.7,44.054075,evergreenForest
BART_031,-71.305095,414.0,44.055533,evergreenForest


In [5]:
#List all species found at each site/plot/date combination
pvtEffort = df.pivot_table(values='daysOfTrapping',
                     index=('siteID','plotID','setDate'),
                     aggfunc="sum",
                     fill_value=0)
dfEffort = pd.DataFrame(pvtEffort)
dfEffort.head()

daysOfTrapping
siteID plotID   setDate                   
BART   BART_002 2014-06-26              56
                2014-07-10              56
                2014-07-24              56
                2014-08-07              56
                2014-08-21              56

In [6]:
#dfEffort['plotID'] = dfEffort.index.levels[1]
dfEff = dfEffort.reset_index()
dfPlot = plotAttributes.reset_index()
dfPlot.columns

Index([u'plotID', u'decimalLongitude', u'elevation', u'decimalLatitude',
       u'nlcdClass'],
      dtype='object')

In [7]:
#Join plot attributes to effort
dfAll = pd.merge(dfEff,dfPlot,how='left',on='plotID')
dfAll.head()

,siteID,plotID,setDate,daysOfTrapping,decimalLongitude,elevation,decimalLatitude,nlcdClass
0,BART,BART_002,2014-06-26,56,-71.27285,550.8,44.035081,deciduousForest
1,BART,BART_002,2014-07-10,56,-71.27285,550.8,44.035081,deciduousForest
2,BART,BART_002,2014-07-24,56,-71.27285,550.8,44.035081,deciduousForest
3,BART,BART_002,2014-08-07,56,-71.27285,550.8,44.035081,deciduousForest
4,BART,BART_002,2014-08-21,56,-71.27285,550.8,44.035081,deciduousForest


In [8]:
dfAll.to_csv(outFN)

In [9]:
dfPin = pd.read_csv('../Scratch/BeetlePinningSummary.csv')
dfPin.columns

Index([u'siteID', u'plotID', u'collectDate', u'Acupalpus partiarius',
       u'Agonoleptus conjunctus', u'Agonoleptus thoracicus', u'Agonum cupreum',
       u'Agonum placidum', u'Agonum retractum', u'Amara (Amara) sp.',
       ...
       u'Stenolophus ochropezus', u'Synuchus impunctatus',
       u'Tetracha carolina', u'Tetracha virginica',
       u'Tetragonoderus intersectus', u'Tetragonoderus laevigatus',
       u'Trechus apicalis', u'Trichotichnus autumnalis',
       u'Trichotichnus fulgens', u'Trichotichnus vulpeculus'],
      dtype='object', length=148)

In [10]:
dfBig = pd.merge(dfAll,dfPin,  how='right', right_on=['plotID','collectDate'],left_on=['plotID','setDate'])
dfBig.to_csv('../Scratch/AllData.csv',index=False)